In [1]:
import os
from other.losses_utils import DistanceCalculator
from other.parsing.train_args_parser import natural_data_mask_saving
import numpy as np
import cv2
from tqdm import tqdm
import torch
from concurrent.futures import ThreadPoolExecutor
from SyntheticData_main.utils import FileManager

In [2]:
save_dir = 'val_coco_dataset'
save_data_masks_dir = (save_dir + '/' + 'masks')
label_dir = os.path.join(save_dir, 'target')

In [3]:
def process_file(filename, label_dir, save_data_masks_dir):
    if not natural_data_mask_saving:
        raise Exception('Natural data mask saving is not enabled.')
    os.makedirs(save_data_masks_dir, exist_ok=True)
    if filename.endswith('.png'):
        image_path = os.path.join(label_dir, filename)
        try:
            img = cv2.imread(image_path)
            if img is None:
                print(f"Could not read {filename}")
                return

            grayscale_tensor = torch.from_numpy(img[:, :, 0]).unsqueeze(0)
            dist_calcul = DistanceCalculator(grayscale_tensor / 255).compute_distance_matrix_on_cpu()
            dist_calcul = dist_calcul.squeeze() * 255.0
            dist_calcul = dist_calcul.numpy().astype(np.uint8)


            FileManager.save_as_grayscale_img(
                dist_calcul,
                os.path.join(save_data_masks_dir, filename)
            )
        except Exception as e:
            print(f"Could not process {filename}: {e}")

In [4]:
def process_masks_in_folder(label_dir, save_data_masks_dir):
    if os.path.exists(label_dir):
        filenames = [f for f in os.listdir(label_dir) if f.endswith('.png')]
        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            list(tqdm(executor.map(lambda f: process_file(f, label_dir, save_data_masks_dir), filenames), desc="Processing files", unit="file", total=len(filenames)))


In [5]:
process_masks_in_folder(label_dir, save_data_masks_dir)

Processing files:   0%|          | 0/2693 [00:00<?, ?file/s]


Exception: Natural data mask saving is not enabled.